# S&P500 Stocks' Jensen's Alpha

## Introduction
##### Stock market is where the stocks are bought and sold, and stock prices regularly fluctuate. Diving deep into the stock price data and modeling the stock returns shows that some of stocks usually show better performance than the others. This notebook answers a very usual question of stock market enthusiasts and practitioners, what type of stocks beat the market?

Notebook is organized in a research paper style. It presents data set and its source, and the usual model for estimation of the Jensen's Alpha. Next, the main results are presented and the notebook is concluded.

##### But first what is the Jensen's Alpha?

Stock prices are volatilem, however, not a single stock can indiciate how the market as a whole is doing. For that reason we look into all of the stocks together for example the index (S&P500, Nikkei225).
So basically stock index shows the whole market itself. If index price is going up that means all the market is doing good.

Historically, individual stocks are tested if they follow the market or not. Usually some of the stocks go up with the market or go down when the market goes up. For that reason several mathematical financial models are developed and the most practical one is called Capital Asset Pricing Model (CAPM).

$$
R_{S} - R_{F} = \beta_{rm} (R_{M} - R_{F})
$$

Here, RS is a stock return, RF is the risk free rate, and RM is the market return. So expectation is that the stock return will relate to the market return by the parameter beta.

However, some of stocks perform better than the market and the CAPM model captures that factor by alpha. It is called Jensen's Alpha (Jensen, 1967., https://papers.ssrn.com/sol3/papers.cfm?abstract_id=244153).

$$
R_{S} - R_{F} = \alpha + \beta_{rm} (R_{M} - R_{F})
$$

Now, stocks with the larger alpha is assumed to perform better than the market and portfolio engineers always look for that alpha.

Thus, in this notebook we will look into S&P500 stocks and estimate the alpha. We also look into which industry is showing better alpha and better market and alpha relation. Since greater the alpha greater the chance of stock performing better than the market.

## Data

Data is coming from two sources. 
FED's open source is used to get risk free rates and Stooq.com is used to get stock prices.

But before, all required packages should be installed.

In [0]:
!pip install datapackage --quiet 

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-35d262a9-03e5-4030-b713-95177aeffc02/bin/python -m pip install --upgrade pip' command.


In [0]:
!pip install pandas_datareader --quiet 

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-35d262a9-03e5-4030-b713-95177aeffc02/bin/python -m pip install --upgrade pip' command.


In [0]:
### import packages 

import datapackage
import pandas as pd
import pandas_datareader as pdr
import statsmodels.api as sm
import numpy as np
np.random.seed(9876789)

### here S&P500 stock names are imported

data_url = "https://datahub.io/core/s-and-p-500-companies/datapackage.json"

package = datapackage.Package(data_url)

resources = package.resources
for resource in resources:
    if resource.tabular:
        data = pd.read_csv(resource.descriptor['path'])


In [0]:
### prepare risk free rate data

rf_data = pdr.get_data_fred('DGS1MO', start = "2010-01-01",end = "2024-01-01")
rf_data["rf"] = rf_data["DGS1MO"] / 100

### lets take a look at the description of the data
### it looks ok since it is already processed and offered through fed source
### min is 0% as expected and max is 6%
### seems like rf data has std of 1.2%

rf_data.describe()

,DGS1MO,rf
count,3405.000000,3405.000000
mean,0.766790,0.007668
std,1.241183,0.012412
min,0.000000,0.000000
25%,0.040000,0.000400
50%,0.110000,0.001100
75%,1.140000,0.011400
max,6.020000,0.060200


In [0]:
### so rf data is from 2010 until 2023
rf_data

,DGS1MO,rf
DATE,,
2010-01-01,NaN,NaN
2010-01-04,0.05,0.0005
2010-01-05,0.03,0.0003
2010-01-06,0.03,0.0003
2010-01-07,0.02,0.0002
...,...,...
2023-08-04,5.54,0.0554
2023-08-07,5.54,0.0554
2023-08-08,5.54,0.0554


In [0]:
#### uncomment this section if it is the first time running the notebook
#### here all stock data is downloaded from stooq

# stock_data_all = pd.DataFrame(index = pd.date_range("2010-01-01","2024-01-01"))

# for ticker in ticker_list[:]:
    
#     stock_data = pdr.get_data_stooq(ticker)
    
#     if stock_data.shape[0] == 0:
#         continue

#     stock_data[ticker + "_close"] = stock_data["Close"].copy()
    
#     stock_data_all = pd.merge(stock_data_all, stock_data[ticker + "_close"], left_index=True, right_index=True, how = "left")

In [0]:
#### if it is the second time running the notebook simply use already downloaded data

# file location and type
file_location = "/FileStore/tables/stck_data-2.csv"
file_type = "csv"

# schema info
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# read data
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# some cleaning and make it pandas df
stock_data_all = df.to_pandas_on_spark()
stock_data_all.set_index("_c0", inplace = True)
stock_data_all.dropna(inplace = True)

stock_data_all = stock_data_all.to_pandas()

/databricks/spark/python/pyspark/sql/dataframe.py:5172: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  warnings.warn(


In [0]:
### lets take a look at the description of the data
### we have all the stock prices for most of stocks in S&P500

stock_data_all.describe()

,MMM_close,AOS_close,ABT_close,ABBV_close,ACN_close,ATVI_close,ADM_close,ADBE_close,AAP_close,AMD_close,...,LRCX_close,LW_close,LVS_close,LEG_close,LDOS_close,LEN_close,LNC_close,LIN_close,LYV_close,LKQ_close
count,842.000000,842.000000,842.000000,842.000000,842.000000,842.000000,842.000000,842.000000,842.000000,842.000000,...,842.000000,842.000000,842.000000,842.000000,842.000000,842.000000,842.000000,842.000000,842.000000,842.000000
mean,145.627096,61.186117,109.435773,121.166836,279.430990,79.948988,66.324533,457.088563,161.186961,90.684804,...,493.721727,75.679435,47.754466,37.181471,95.424131,88.063281,46.685898,289.769903,80.696900,45.959838
std,27.171471,10.120109,10.909956,27.511840,47.378502,8.467008,17.816031,97.277947,38.906535,22.851178,...,116.234307,16.797236,9.087701,5.772118,7.143269,17.942782,15.736039,49.610363,20.022655,10.596039
min,93.310000,36.332000,78.450500,66.494100,158.022000,56.942000,31.611200,275.200000,64.511200,47.520000,...,226.774000,49.483700,30.140000,21.949800,76.829800,37.733500,19.110000,168.683000,34.510000,19.364800
25%,125.016500,53.698350,103.066500,98.541200,251.318500,75.768250,51.388775,377.025000,139.498250,77.457500,...,419.520250,62.650500,38.975000,32.974975,90.289500,75.878150,32.504625,250.173000,69.142500,37.014025
50%,145.236000,62.082900,108.626000,116.876500,280.282000,78.875500,65.204500,453.805000,160.527500,87.150000,...,500.191500,74.019800,47.090000,36.644350,96.052100,88.088900,47.408950,293.360000,81.780000,49.470300
75%,168.140500,68.360300,116.659000,146.597500,312.266250,83.519250,81.946750,508.205000,192.440250,105.270000,...,588.529500,82.376325,56.527500,40.467650,100.865000,100.811000,61.588725,325.897250,91.932500,54.330000
max,194.591000,84.268300,139.123000,170.151000,409.834000,102.700000,97.670000,688.370000,229.551000,161.910000,...,721.260000,115.120000,66.200000,53.726800,110.117000,133.240000,73.727600,391.040000,126.040000,59.010000


In [0]:
### our stock price data is in a daily frequency and it is from 2020 April until 2023 August
stock_data_all

,MMM_close,AOS_close,ABT_close,ABBV_close,ACN_close,ATVI_close,ADM_close,ADBE_close,AAP_close,AMD_close,...,LRCX_close,LW_close,LVS_close,LEG_close,LDOS_close,LEN_close,LNC_close,LIN_close,LYV_close,LKQ_close
_c0,,,,,,,,,,,,,,,,,,,,,
2020-04-06,127.274,36.3320,79.2164,66.7952,159.878,60.543,33.4236,319.13,89.1787,47.52,...,242.462,49.7015,40.70,22.5810,91.2191,37.7335,24.5451,168.683,34.51,19.5797
2020-04-07,130.790,37.7006,78.4505,66.4941,158.022,58.842,33.6090,308.93,92.5583,47.56,...,247.753,52.8480,42.70,23.6767,89.0183,37.7731,26.7544,172.540,37.85,20.4382
2020-04-08,134.781,38.2330,81.3439,69.2941,165.354,60.022,34.0766,317.18,95.4183,48.79,...,257.931,55.3901,46.55,25.6483,91.7483,41.7168,29.6094,179.244,37.76,21.4732
2020-04-09,133.686,38.2232,82.3884,70.3415,171.296,59.423,35.1027,318.70,99.4560,48.38,...,249.752,56.0124,47.86,26.6369,91.7681,43.1033,31.5723,181.676,38.27,21.9907
2020-04-13,132.436,37.6729,82.2040,70.8261,166.137,61.673,33.6928,320.65,102.3750,50.94,...,250.226,55.8783,46.49,25.5683,88.5177,39.7845,29.8047,178.452,39.41,20.8286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-07,105.230,73.8400,108.4500,150.3300,315.870,91.570,87.2100,529.73,71.3000,116.81,...,705.360,103.1000,58.14,29.6900,97.6300,127.3100,27.6500,384.970,89.22,54.2300
2023-08-08,104.120,73.2400,105.5800,149.6200,315.150,91.590,86.4400,520.60,70.5600,113.23,...,696.000,100.4200,57.52,29.4900,97.0000,127.0300,27.5800,381.780,87.88,53.3150
2023-08-09,103.710,72.7900,105.3000,150.7700,311.530,91.440,85.8500,513.78,71.2000,110.47,...,686.050,99.3300,56.85,29.5900,97.5600,125.5800,26.6400,379.560,86.30,53.3600


In [0]:
### data preparation
### prepare S&P500 and risk free data

market_data = pdr.get_data_fred('SP500')
market_data["market_return"] = market_data["SP500"].diff() / market_data["SP500"]

rf_data = pdr.get_data_fred('DGS1MO', start = "2010-01-01",end = "2024-01-01")
rf_data["rf"] = rf_data["DGS1MO"] / 100
rf_data["yyyymm"] = rf_data.index.year.astype("str") + rf_data.index.month.astype("str").str.zfill(2)


## Modeling

First we start by fitting a linear regression. By using Ordinary Least Squares method we estimate parameters. They are estimated parameters so they will have a cap on it.

$$
R_{i,t} - R_{F} = \hat{\alpha}_{i} + \hat{\beta}_{i} (R_{M}  - R_{F} )+ \epsilon_{i,t}
$$

Here, i is for stock ticker, and t is for time.

So now we have market index price data, risk free data and stock prices. We can start modeling from now on.

But before that lets choose one stock and explain what is the model result and what is the estimated alpha.

In [0]:
## lets choose apple stocks
ticker = "AAPL"

## make temporary data
stock_data = stock_data_all[[ticker + "_close"]].copy()

## estimate returns
stock_data["stock_return"] = stock_data[ticker + "_close"].diff() / stock_data[ticker + "_close"]

## merge with market data
data_mr = pd.merge(market_data["market_return"], 
                   stock_data["stock_return"], 
                   left_index=True, 
                   right_index=True)

data_mr["yyyymm"] = data_mr.index.year.astype("str") + data_mr.index.month.astype("str").str.zfill(2)
data_mr["yyyymmdd"] = data_mr.index.copy()

## merge with rf data
data_mr = pd.merge(data_mr[["yyyymmdd","stock_return","market_return","yyyymm"]], 
                   rf_data[["rf","yyyymm"]], 
                   left_on="yyyymm", 
                   right_on="yyyymm")

data_mr["stock_return"] = data_mr["stock_return"] - data_mr["rf"] 
data_mr["market_return"] = data_mr["market_return"] - data_mr["rf"] 

## clean data
data_mr.dropna(inplace=True)

## add constant into the data so model will estimate alpha
data_mr = sm.add_constant(data_mr)

## model fitting
model = sm.OLS(data_mr.stock_return, data_mr[["const","market_return"]])
results = model.fit()


In [0]:
print(results.summary())

### here R-squared shows that mraket can explain almost 80% (0.789) of the fluctuations in apple stocks
### looking into coef for const (that is our alpha) we see P value is almost zero
### p value shows if the coef is statistically significant
### so in short, apple stocks showing 0.0015 better performance than the market

                            OLS Regression Results                            
Dep. Variable:           stock_return   R-squared:                       0.789
Model:                            OLS   Adj. R-squared:                  0.789
Method:                 Least Squares   F-statistic:                 6.301e+04
Date:                Sun, 13 Aug 2023   Prob (F-statistic):               0.00
Time:                        13:41:58   Log-Likelihood:                 49905.
No. Observations:               16835   AIC:                        -9.981e+04
Df Residuals:                   16833   BIC:                        -9.979e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0015      0.000     13.636

In [0]:
### here we repeat the same process for all other stocks 
### estimate the alpha and save them for further use

In [0]:
ticker_list = data["Symbol"].tolist()

market_pv_list = list()
alpha_pv_list = list()

market_coef_list = list()
alpha_coef_list = list()

symbol_list = list()
    
for ticker in ticker_list[:]:
    
    if ticker + "_close" not in stock_data_all.columns.tolist():
        continue

    stock_data = stock_data_all[[ticker + "_close"]].copy()

    if stock_data.shape[0] == 0:
        continue

    stock_data["stock_return"] = stock_data[ticker + "_close"].diff() / stock_data[ticker + "_close"]

    ## merge with market data
    data_mr = pd.merge(market_data["market_return"], 
                       stock_data["stock_return"], 
                       left_index=True, 
                       right_index=True)

    data_mr["yyyymm"] = data_mr.index.year.astype("str") + data_mr.index.month.astype("str").str.zfill(2)
    data_mr["yyyymmdd"] = data_mr.index.copy()

    ## merge with rf data
    data_mr = pd.merge(data_mr[["yyyymmdd","stock_return","market_return","yyyymm"]], 
                       rf_data[["rf","yyyymm"]], 
                       left_on="yyyymm", 
                       right_on="yyyymm")


    data_mr["stock_return"] = data_mr["stock_return"] - data_mr["rf"] 
    data_mr["market_return"] = data_mr["market_return"] - data_mr["rf"] 

    ## clean data
    data_mr.dropna(inplace=True)

    ## add constant into the data so model will estimate alpha
    data_mr = sm.add_constant(data_mr)

    ## model fitting
    model = sm.OLS(data_mr.stock_return, data_mr[["const","market_return"]])
    results = model.fit()

    ## save parameters
    market_pv_list.append(results.pvalues[0]) ##market pvalue
    alpha_pv_list.append(results.pvalues[1]) ##alpha pvalue

    ## save pvalues
    market_coef_list.append(results.params[0]) ##market pvalue
    alpha_coef_list.append(results.params[1]) ##alpha pvalue

    ## save tickers
    symbol_list.append(ticker)


In [0]:
### here save all data into dataframe
final_data = pd.DataFrame()
final_data["symbol"] = symbol_list
final_data["alpha_coef"] = alpha_coef_list
final_data["alpha_pv"] = alpha_pv_list
final_data["market_coef"] = market_coef_list
final_data["marke_pv"] = market_pv_list

### lets focus on only significant data
final_data_significant = final_data[final_data["alpha_pv"] <= 0.1]

## Results

### Estimated Jensen's alpha

In [0]:
## lets check alphas
## it seems it goes up to 1.35 and goes down to 1

final_data.describe()

,alpha_coef,alpha_pv,market_coef,marke_pv
count,272.000000,272.0,272.000000,2.720000e+02
mean,1.003836,0.0,-0.000077,6.896246e-02
std,0.100255,0.0,0.001489,2.088660e-01
min,0.781651,0.0,-0.003749,4.270288e-139
25%,0.940413,0.0,-0.001065,8.226774e-27
50%,0.994910,0.0,-0.000081,1.222609e-10
75%,1.067934,0.0,0.000902,9.180182e-04
max,1.352819,0.0,0.004972,9.948215e-01


In [0]:
### by looking into all alpha in below graph we see interesting result

### just for reminder the meaning of alpha is, higher the alpha from 1 better the performance and alphas lower than 1 are the cases when stock is doing poorly than the market

### thus we can conclude that stocks forming two groups
### first, over 1 alpha. they do better than the market return
### second is the lower than 1 alpha. they usually doing worse than the market

### next we check market coef

In [0]:
display(final_data_significant)

symbol,alpha_coef,alpha_pv,market_coef,marke_pv
MMM,0.9427178083213479,0.0,-0.0015539360426961371,1.670622229561558E-38
AOS,0.9832066810561861,0.0,-1.8329952352246748E-4,0.17999632812561028
ABT,0.9437416800069042,0.0,-9.724477885143035E-4,3.3920310773315196E-18
ABBV,0.8681055476238482,0.0,-0.0014565050303953363,7.350443181364701E-29
ACN,1.0582471024393767,0.0,9.112481522191239E-4,5.303410964462857E-22
ATVI,0.8256361527697422,0.0,-0.002733113392205081,5.688208762751674E-72
ADM,0.9525470565469555,0.0,-1.5889983654266158E-4,0.23008884106774014
ADBE,1.073428172095016,0.0,8.681931616285828E-4,5.998061637252829E-8
AAP,1.090304304910073,0.0,-6.1123700666051E-5,0.790372371306961
AMD,1.1849455431520013,0.0,0.0023017699328884414,4.548620447868185E-22


Databricks visualization. Run in Databricks to view.

In [0]:
### stocks's market relation is captured by the beta factor
### interestingly some of stocks are showing strong relation to the overall market while other stocks are showing less then 0 relation

### but most imporantly as we expected stocks vary depending on how they relate to the market
### we can see clearly two groups are formed, one positive, higher than 0 market coef, and the other group of stocks have negative market coef.
### stocks with positive coef will go up when the market goes up and stocks with negative coef will go down when market goes up


In [0]:
display(final_data_significant)

symbol,alpha_coef,alpha_pv,market_coef,marke_pv
MMM,0.9427178083213479,0.0,-0.0015539360426961371,1.670622229561558E-38
AOS,0.9832066810561861,0.0,-1.8329952352246748E-4,0.17999632812561028
ABT,0.9437416800069042,0.0,-9.724477885143035E-4,3.3920310773315196E-18
ABBV,0.8681055476238482,0.0,-0.0014565050303953363,7.350443181364701E-29
ACN,1.0582471024393767,0.0,9.112481522191239E-4,5.303410964462857E-22
ATVI,0.8256361527697422,0.0,-0.002733113392205081,5.688208762751674E-72
ADM,0.9525470565469555,0.0,-1.5889983654266158E-4,0.23008884106774014
ADBE,1.073428172095016,0.0,8.681931616285828E-4,5.998061637252829E-8
AAP,1.090304304910073,0.0,-6.1123700666051E-5,0.790372371306961
AMD,1.1849455431520013,0.0,0.0023017699328884414,4.548620447868185E-22


Databricks visualization. Run in Databricks to view.

In [0]:
### thus so far our main results
### alphas form two groups
### market relation also forms two groups

### thus we need to check by industry wise averages
### different type of industry should have different alpha


In [0]:
### lets look into data by industry wise
### we look into average alphas for each industry

data_final_mr = pd.merge(final_data, data, left_on="symbol", right_on="Symbol")
data_final_mr_gr = data_final_mr.groupby("Sector").mean()
data_final_mr_gr.reset_index(inplace=True)

In [0]:
### so on average energy, information technology, financials and consumer discretionary stocks are showing better alpha
display(data_final_mr_gr)

Sector,alpha_coef,alpha_pv,market_coef,marke_pv
Communication Services,0.9887690260119372,0.0,-6.176347960536284E-4,0.0027453227350436975
Consumer Discretionary,1.0660359552394278,0.0,7.130931998092277E-4,0.1255952307271121
Consumer Staples,0.8644323607738976,0.0,-0.002059448748294051,0.012329089785414438
Energy,1.0392656469313688,0.0,0.0012006994441430706,0.08147320743036267
Financials,1.0398462677077265,0.0,4.11376681554383E-4,0.06947837807314658
Health Care,0.9600701443875963,0.0,-7.720224840779571E-4,0.030317481861112907
Industrials,1.0004097043336977,0.0,2.239521831201821E-5,0.100591288235162
Information Technology,1.0658617440040925,0.0,7.647665388793393E-4,0.0945505805499622
Materials,1.0269500752596488,0.0,3.9018914396940957E-4,0.0871045796832961
Real Estate,1.0034779038531958,0.0,-3.6672223145153984E-4,0.06995400999691269


Databricks visualization. Run in Databricks to view.

In [0]:
### lets check the alpha and market relation in one graph
### this combination should find us the best performing sector
### diving deep into that shows that the energy sector shows the best combinations
### meaining that it has stronger relation with the market itself and also higher alpha

display(data_final_mr_gr)

Sector,alpha_coef,alpha_pv,market_coef,marke_pv
Communication Services,0.9887690260119372,0.0,-6.176347960536284E-4,0.0027453227350436975
Consumer Discretionary,1.0660359552394278,0.0,7.130931998092277E-4,0.1255952307271121
Consumer Staples,0.8644323607738976,0.0,-0.002059448748294051,0.012329089785414438
Energy,1.0392656469313688,0.0,0.0012006994441430706,0.08147320743036267
Financials,1.0398462677077265,0.0,4.11376681554383E-4,0.06947837807314658
Health Care,0.9600701443875963,0.0,-7.720224840779571E-4,0.030317481861112907
Industrials,1.0004097043336977,0.0,2.239521831201821E-5,0.100591288235162
Information Technology,1.0658617440040925,0.0,7.647665388793393E-4,0.0945505805499622
Materials,1.0269500752596488,0.0,3.9018914396940957E-4,0.0871045796832961
Real Estate,1.0034779038531958,0.0,-3.6672223145153984E-4,0.06995400999691269


Databricks visualization. Run in Databricks to view.

### Industry return

Since we found based on alpha and market relation the high performing industry, now it is time to actually dive deeper into industry returns.
As we saw earlier, energy sector shows the best combination of alpha and beta relation. So we assume energy sector return should be the best.

In [0]:
### construct portfolio of those high performing energy sectors

In [0]:
stock_data_all_v1 = pd.DataFrame((stock_data_all.loc[stock_data_all.index[-1]] - stock_data_all.loc[stock_data_all.index[0]]) / stock_data_all.loc[stock_data_all.index[0]], columns=["return"])
stock_data_all_v1.index = [a.split("_")[0] for a in stock_data_all_v1.index.tolist()]
stock_data_all_v1.reset_index(inplace = True)
stock_data_all_v1.rename(columns={"index":"symbol"}, inplace=True)

In [0]:
data_mr_v1 = pd.merge(data, stock_data_all_v1, left_on="Symbol", right_on="symbol")
data_mr_v1 = data_mr_v1.groupby("Sector").mean()
data_mr_v1.reset_index(inplace=True)

In [0]:
data_mr_v1.sort_values("return")

,Sector,return
10,Utilities,0.243739
2,Consumer Staples,0.415121
9,Real Estate,0.424422
0,Communication Services,0.521737
5,Health Care,0.591387
4,Financials,0.909714
6,Industrials,0.969772
7,Information Technology,0.991691
1,Consumer Discretionary,1.044476
8,Materials,1.126363


In [0]:
### based on results we see that energy sector indeed is showing the best return
### also materials and consumer discretionary also have high returns

In [0]:
display(data_mr_v1.sort_values("return"))

Sector,return
Utilities,0.24373865959411992
Consumer Staples,0.4151205199386811
Real Estate,0.424421898970612
Communication Services,0.5217368446832756
Health Care,0.5913869372533193
Financials,0.9097141694184505
Industrials,0.9697719584314548
Information Technology,0.9916908284375117
Consumer Discretionary,1.0444758167761303
Materials,1.1263629562645865


Databricks visualization. Run in Databricks to view.

## Conclusion

In this notebook we have seen how to estimate Jensen's alpha, a simple indicator that shows which stocks are doing better than the market.

By looking into almost 500 stocks we estimated the alphas and checked the best stock sector based on the alpha and beta relation.

Looking into that relation we found that indeed best sector that performed well for the past years is the energy sector stocks.